In [2]:
import torch 
torch.__version__, torch.cuda.is_available()

('1.7.1', True)

In [ ]:
import os
import matplotlib
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
root = '/home/.faa'
data_path = os.path.join(root, 'data')
def cifar10_data(train, batch_size):
    cifar10 = datasets.CIFAR10(data_path, train=train, transform=transforms.ToTensor())
    loader = DataLoader(cifar10, batch_size=batch_size, shuffle=True,)
    return loader
data = cifar10_data(False, 128)

In [ ]:
import torchvision
torchvision.__version__

In [ ]:
from robustbench.utils import load_model
model_path = os.path.join(root, 'model')
model = load_model(model_name='Carmon2019Unlabeled', model_dir=model_path, dataset='cifar10',threat_model='Linf')

In [ ]:
import numpy as np
num_trials = 10
for x, y in data:
    x_hsv = matplotlib.colors.rgb_to_hsv(x)
    for i in range(num_trials):
        # Randomly shift Hue and Saturation components
        X_adv_hsv = np.copy(X_hsv)
        d_h = np.random.uniform(0, 1, size=(X_adv_hsv.shape[0],1))
        d_s = np.random.uniform(-1, 1, size=(X_adv_hsv.shape[0],1)) * float(i) / num_trials

        for j in range(X_adv_hsv.shape[0]):
            X_adv_hsv[j, :, :, 0] = (X_hsv[j, :, :, 0] + d_h[j]) % 1.0
            X_adv_hsv[j, :, :, 1] = np.clip(X_hsv[j, :, :, 1] + d_s[j], 0., 1.)

        X = matplotlib.colors.hsv_to_rgb(X_adv_hsv)
        X = np.clip(X, 0., 1.)

        # extract out wrongly-classified images

        pred_label = np.argmax(model.predict(data_normalize(X * 255.)), axis=1)
        wrong_labels = pred_label != y.reshape(-1)
        
        X_adv.append(X[wrong_labels]) # store wrongly-classified images

        X_hsv = X_hsv[wrong_labels == 0]
        y = y[wrong_labels == 0]

        adv_succ_num[i + 1] = np.sum(wrong_labels)

        print('Trial ' + str(i+1) +
              ', Attack success rate: ' + str(np.sum(adv_succ_num)/N))

    return X_adv, adv_succ_num